In [ ]:
import os 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 
import math
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelWithLMHead , AutoModelForSeq2SeqLM

In [ ]:
tokenizer_hindi = AutoTokenizer.from_pretrained("rahular/varta-t5")
model_hindi = AutoModelForSeq2SeqLM.from_pretrained("rahular/varta-t5")

def summarize_transformer_hindi(sentences):
    inputs=tokenizer_hindi.encode("summarize: " +"".join(sentences),return_tensors='pt', max_length=512, truncation=True)
    output = model_hindi.generate(inputs, min_length=80, max_length=100)
    summary=tokenizer_hindi.decode(output[0])
    summary=summary.replace('<pad>','')
    summary=summary.replace('</s>','')
    summary=summary.replace('<unk>','<')
    summary= summary.strip(' ')
    return summary

In [ ]:
input_path="../data/"
output_path="../results/summary/"
os.makedirs(output_path, exist_ok=True)
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

data = pd.read_csv(f'{input_path}data.csv')
sentences = data['sentences'].apply(eval)

summary=[]
for i in tqdm(range(len(sentences))):
    summary.append(summarize_transformer_hindi( sentences[i]))

data['summary'] = summary

data.to_csv(f'{output_path}transformer.csv', index=False)